<a href="https://colab.research.google.com/github/01star01ek/01star01ek/blob/main/%EC%B5%9C%EC%A2%85%EC%9A%B0%EC%8A%B9%EC%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install insightface onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1060438 sha256=f616035843089a8e68d930ae39e6db694ed463929ee69e7784c1558e6e910ca0
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface


In [5]:
import requests
import os

def download_inswapper():
    url = "https://civitai.com/api/download/models/85159"

    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open('inswapper_128.onnx', 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("모델 다운로드 완료!")
        return True
    else:
        print(f"다운로드 실패: {response.status_code}")
        return False

download_inswapper()

모델 다운로드 완료!


True

In [14]:
import cv2
import insightface
from insightface.app import FaceAnalysis
import os

# 설정
app = FaceAnalysis()
app.prepare(ctx_id=0, det_size=(320, 320))
swapper = insightface.model_zoo.get_model('inswapper_128.onnx')

# 소스 이미지
source_img = cv2.imread('source.png')
source_faces = app.get(source_img)
source_face = source_faces[0]

# 비디오 처리
cap = cv2.VideoCapture('target.mp4')
os.makedirs('output', exist_ok=True)

frame_count = 0
saved_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 10프레임마다 처리
    if frame_count % 10 == 0:
        target_faces = app.get(frame)

        if len(target_faces) > 0:
            # 얼굴 합성
            result = swapper.get(frame, target_faces[0], source_face, paste_back=True)

            # 이미지 저장
            cv2.imwrite(f'output/frame_{saved_count:04d}.png', result)
            saved_count += 1
            print(f"저장: frame_{saved_count:04d}.png")

    frame_count += 1

cap.release()
print(f"완료! {saved_count}개 이미지 저장")

/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (320, 320)
Applied prov

In [18]:
from glob import glob
print("영상 생성 중...")

# 이미지 파일들 가져오기
image_files = sorted(glob('output/frame_*.png'))
if len(image_files) > 0:
    # 첫 번째 이미지로 크기 확인
    first_img = cv2.imread(image_files[0])
    height, width, _ = first_img.shape

    # 비디오 라이터 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('final_result.mp4', fourcc, 30, (width, height))

    # 모든 이미지를 영상에 추가
    for img_path in image_files:
        img = cv2.imread(img_path)
        out.write(img)

    out.release()
    print(f"✅ 영상 완성! final_result.mp4 ({len(image_files)}개 프레임)")
else:
    print("❌ 저장된 이미지가 없습니다!")

영상 생성 중...
✅ 영상 완성! final_result.mp4 (36개 프레임)


In [20]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=2a6f2be9ec541a3f655187985f9e59a500127d65d8fdc4b36dab8da35750079c
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [22]:
import cv2
import numpy as np
import time
import subprocess
import os
import json
import matplotlib.pyplot as plt
from datetime import datetime
import psutil
import GPUtil
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import warnings
warnings.filterwarnings('ignore')

class FrameInterpolationBenchmark:
    def __init__(self, input_video_path, output_dir="benchmark_results"):
        """
        프레임 보간 성능 비교 평가 시스템

        Args:
            input_video_path: 테스트할 입력 영상 경로
            output_dir: 결과 저장 디렉토리
        """
        self.input_video = input_video_path
        self.output_dir = output_dir
        self.results = {}

        # 출력 디렉토리 생성
        os.makedirs(output_dir, exist_ok=True)

        # 영상 정보 가져오기
        self.get_video_info()

        print(f"🎬 입력 영상: {input_video_path}")
        print(f"📊 영상 정보: {self.video_info}")

    def get_video_info(self):
        """입력 영상 정보 분석"""
        cap = cv2.VideoCapture(self.input_video)

        self.video_info = {
            'fps': cap.get(cv2.CAP_PROP_FPS),
            'width': int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            'height': int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
            'frame_count': int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
            'duration': cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
        }

        cap.release()

    def monitor_resources(self):
        """시스템 리소스 모니터링"""
        cpu_percent = psutil.cpu_percent(interval=1)
        memory = psutil.virtual_memory()

        gpu_info = []
        try:
            gpus = GPUtil.getGPUs()
            for gpu in gpus:
                gpu_info.append({
                    'name': gpu.name,
                    'utilization': gpu.load * 100,
                    'memory_used': gpu.memoryUsed,
                    'memory_total': gpu.memoryTotal
                })
        except:
            gpu_info = [{'name': 'N/A', 'utilization': 0, 'memory_used': 0, 'memory_total': 0}]

        return {
            'cpu_percent': cpu_percent,
            'memory_percent': memory.percent,
            'memory_used_gb': memory.used / (1024**3),
            'gpu_info': gpu_info
        }

    def test_ffmpeg_minterpolate(self, target_fps=60):
        """FFmpeg minterpolate 테스트"""
        print(f"\n🔄 FFmpeg minterpolate 테스트 (목표 FPS: {target_fps})")

        output_path = os.path.join(self.output_dir, f"ffmpeg_minterpolate_{target_fps}fps.mp4")

        start_time = time.time()
        start_resources = self.monitor_resources()

        cmd = [
            'ffmpeg', '-i', self.input_video,
            '-filter:v', f'minterpolate=fps={target_fps}:mi_mode=mci',
            '-y', output_path
        ]

        try:
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
            processing_time = time.time() - start_time

            if result.returncode == 0:
                end_resources = self.monitor_resources()
                file_size = os.path.getsize(output_path) / (1024**2)  # MB

                self.results['FFmpeg_minterpolate'] = {
                    'status': 'success',
                    'processing_time': processing_time,
                    'output_path': output_path,
                    'file_size_mb': file_size,
                    'start_resources': start_resources,
                    'end_resources': end_resources,
                    'target_fps': target_fps
                }
                print(f"✅ 완료! 처리시간: {processing_time:.2f}초, 파일크기: {file_size:.2f}MB")
            else:
                print(f"❌ 실패: {result.stderr}")
                self.results['FFmpeg_minterpolate'] = {'status': 'failed', 'error': result.stderr}

        except subprocess.TimeoutExpired:
            print("❌ 시간 초과 (300초)")
            self.results['FFmpeg_minterpolate'] = {'status': 'timeout'}
        except Exception as e:
            print(f"❌ 오류: {e}")
            self.results['FFmpeg_minterpolate'] = {'status': 'error', 'error': str(e)}

    def test_rife(self, multiplier=2):
        """RIFE 테스트 (GitHub에서 클론 필요)"""
        print(f"\n🔄 RIFE 테스트 (multiplier: {multiplier})")

        rife_dir = "ECCV2022-RIFE"
        if not os.path.exists(rife_dir):
            print("⚠️  RIFE 저장소 클론 중...")
            subprocess.run(['git', 'clone', 'https://github.com/megvii-research/ECCV2022-RIFE.git'])

        output_path = os.path.join(self.output_dir, f"rife_x{multiplier}.mp4")

        start_time = time.time()
        start_resources = self.monitor_resources()

        cmd = [
            'python', f'{rife_dir}/inference_video.py',
            f'--exp={multiplier}',
            f'--video={self.input_video}',
            f'--output={output_path}'
        ]

        try:
            os.chdir(rife_dir)
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=600)
            os.chdir('..')

            processing_time = time.time() - start_time

            if result.returncode == 0 and os.path.exists(output_path):
                end_resources = self.monitor_resources()
                file_size = os.path.getsize(output_path) / (1024**2)

                self.results['RIFE'] = {
                    'status': 'success',
                    'processing_time': processing_time,
                    'output_path': output_path,
                    'file_size_mb': file_size,
                    'start_resources': start_resources,
                    'end_resources': end_resources,
                    'multiplier': multiplier
                }
                print(f"✅ 완료! 처리시간: {processing_time:.2f}초, 파일크기: {file_size:.2f}MB")
            else:
                print(f"❌ 실패: {result.stderr}")
                self.results['RIFE'] = {'status': 'failed', 'error': result.stderr}

        except subprocess.TimeoutExpired:
            print("❌ 시간 초과 (600초)")
            self.results['RIFE'] = {'status': 'timeout'}
        except Exception as e:
            print(f"❌ 오류: {e}")
            self.results['RIFE'] = {'status': 'error', 'error': str(e)}

    def test_frame_blending(self, target_fps=60):
        """간단한 프레임 블렌딩 테스트 (기준선)"""
        print(f"\n🔄 프레임 블렌딩 테스트 (목표 FPS: {target_fps})")

        output_path = os.path.join(self.output_dir, f"frame_blending_{target_fps}fps.mp4")

        start_time = time.time()
        start_resources = self.monitor_resources()

        try:
            cap = cv2.VideoCapture(self.input_video)

            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, target_fps,
                                (self.video_info['width'], self.video_info['height']))

            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)

            cap.release()

            # 프레임 간 블렌딩으로 보간
            interpolation_factor = target_fps / self.video_info['fps']

            for i in range(len(frames) - 1):
                out.write(frames[i])

                # 중간 프레임들 생성
                num_inter_frames = int(interpolation_factor) - 1
                for j in range(1, num_inter_frames + 1):
                    alpha = j / (num_inter_frames + 1)
                    blended = cv2.addWeighted(frames[i], 1-alpha, frames[i+1], alpha, 0)
                    out.write(blended)

            if frames:
                out.write(frames[-1])

            out.release()

            processing_time = time.time() - start_time
            end_resources = self.monitor_resources()
            file_size = os.path.getsize(output_path) / (1024**2)

            self.results['Frame_Blending'] = {
                'status': 'success',
                'processing_time': processing_time,
                'output_path': output_path,
                'file_size_mb': file_size,
                'start_resources': start_resources,
                'end_resources': end_resources,
                'target_fps': target_fps
            }
            print(f"✅ 완료! 처리시간: {processing_time:.2f}초, 파일크기: {file_size:.2f}MB")

        except Exception as e:
            print(f"❌ 오류: {e}")
            self.results['Frame_Blending'] = {'status': 'error', 'error': str(e)}

    def calculate_quality_metrics(self, method_name, output_path):
        """품질 지표 계산 (PSNR, SSIM)"""
        print(f"\n📊 {method_name} 품질 분석...")

        try:
            # 원본과 결과 영상에서 샘플 프레임 추출
            original_frames = self.extract_sample_frames(self.input_video, num_samples=10)
            result_frames = self.extract_sample_frames(output_path, num_samples=10)

            if len(original_frames) == 0 or len(result_frames) == 0:
                return None

            psnr_scores = []
            ssim_scores = []

            # 크기 맞추기
            min_samples = min(len(original_frames), len(result_frames))

            for i in range(min_samples):
                orig = original_frames[i]
                result = result_frames[i]

                # 크기 조정
                if orig.shape != result.shape:
                    result = cv2.resize(result, (orig.shape[1], orig.shape[0]))

                # 그레이스케일 변환
                orig_gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
                result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

                # PSNR 계산
                psnr_val = psnr(orig_gray, result_gray, data_range=255)
                psnr_scores.append(psnr_val)

                # SSIM 계산
                ssim_val = ssim(orig_gray, result_gray, data_range=255)
                ssim_scores.append(ssim_val)

            quality_metrics = {
                'avg_psnr': np.mean(psnr_scores),
                'avg_ssim': np.mean(ssim_scores),
                'psnr_std': np.std(psnr_scores),
                'ssim_std': np.std(ssim_scores)
            }

            self.results[method_name]['quality_metrics'] = quality_metrics
            print(f"   PSNR: {quality_metrics['avg_psnr']:.2f} ± {quality_metrics['psnr_std']:.2f}")
            print(f"   SSIM: {quality_metrics['avg_ssim']:.3f} ± {quality_metrics['ssim_std']:.3f}")

            return quality_metrics

        except Exception as e:
            print(f"❌ 품질 분석 실패: {e}")
            return None

    def extract_sample_frames(self, video_path, num_samples=10):
        """영상에서 샘플 프레임 추출"""
        cap = cv2.VideoCapture(video_path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if frame_count == 0:
            cap.release()
            return []

        frames = []
        step = max(1, frame_count // num_samples)

        for i in range(0, frame_count, step):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if ret:
                frames.append(frame)
                if len(frames) >= num_samples:
                    break

        cap.release()
        return frames

    def run_all_tests(self):
        """모든 테스트 실행"""
        print("🚀 프레임 보간 성능 비교 테스트 시작!")
        print("=" * 60)

        # 1. FFmpeg minterpolate
        self.test_ffmpeg_minterpolate(target_fps=60)

        # 2. Frame Blending (기준선)
        self.test_frame_blending(target_fps=60)

        # 3. RIFE (조건부)
        if self.should_test_rife():
            self.test_rife(multiplier=2)
        else:
            print("\n⚠️  RIFE 테스트 건너뜀 (의존성 없음)")

        # 품질 분석
        print("\n" + "=" * 60)
        print("📊 품질 분석 시작...")

        for method, result in self.results.items():
            if result.get('status') == 'success':
                self.calculate_quality_metrics(method, result['output_path'])

        # 결과 저장 및 시각화
        self.save_results()
        self.create_comparison_report()

    def should_test_rife(self):
        """RIFE 테스트 가능 여부 확인"""
        try:
            # PyTorch 설치 여부 확인
            import torch
            return True
        except ImportError:
            return False

    def save_results(self):
        """결과를 JSON 파일로 저장"""
        results_file = os.path.join(self.output_dir, "benchmark_results.json")

        # datetime 객체들을 문자열로 변환
        serializable_results = {}
        for method, result in self.results.items():
            serializable_results[method] = {}
            for key, value in result.items():
                if isinstance(value, (dict, list, str, int, float, bool)) or value is None:
                    serializable_results[method][key] = value
                else:
                    serializable_results[method][key] = str(value)

        with open(results_file, 'w', encoding='utf-8') as f:
            json.dump({
                'timestamp': datetime.now().isoformat(),
                'input_video_info': self.video_info,
                'results': serializable_results
            }, f, indent=2, ensure_ascii=False)

        print(f"💾 결과 저장: {results_file}")

    def create_comparison_report(self):
        """비교 리포트 생성"""
        print("\n" + "=" * 60)
        print("📊 최종 비교 리포트")
        print("=" * 60)

        successful_methods = {k: v for k, v in self.results.items() if v.get('status') == 'success'}

        if not successful_methods:
            print("❌ 성공한 테스트가 없습니다!")
            return

        # 성능 비교 테이블
        print(f"{'방법':<20} {'처리시간(초)':<12} {'파일크기(MB)':<12} {'PSNR':<8} {'SSIM':<8}")
        print("-" * 70)

        for method, result in successful_methods.items():
            processing_time = result.get('processing_time', 0)
            file_size = result.get('file_size_mb', 0)

            quality = result.get('quality_metrics', {})
            psnr_val = quality.get('avg_psnr', 0)
            ssim_val = quality.get('avg_ssim', 0)

            print(f"{method:<20} {processing_time:<12.2f} {file_size:<12.2f} {psnr_val:<8.2f} {ssim_val:<8.3f}")

        # 추천 결과
        print("\n🏆 추천 결과:")

        # 속도 기준 최고
        fastest = min(successful_methods.items(), key=lambda x: x[1].get('processing_time', float('inf')))
        print(f"   ⚡ 가장 빠름: {fastest[0]} ({fastest[1].get('processing_time', 0):.2f}초)")

        # 품질 기준 최고 (SSIM)
        quality_methods = {k: v for k, v in successful_methods.items() if v.get('quality_metrics')}
        if quality_methods:
            best_quality = max(quality_methods.items(),
                             key=lambda x: x[1].get('quality_metrics', {}).get('avg_ssim', 0))
            ssim_score = best_quality[1].get('quality_metrics', {}).get('avg_ssim', 0)
            print(f"   🎯 최고 품질: {best_quality[0]} (SSIM: {ssim_score:.3f})")

        # 파일 크기 최적화
        smallest = min(successful_methods.items(), key=lambda x: x[1].get('file_size_mb', float('inf')))
        print(f"   💾 최소 용량: {smallest[0]} ({smallest[1].get('file_size_mb', 0):.2f}MB)")

# 사용 예시
def main():
    # 테스트할 영상 경로
    input_video = "final_result.mp4"  # 여기에 테스트할 영상 경로 입력

    if not os.path.exists(input_video):
        print(f"❌ 입력 영상을 찾을 수 없습니다: {input_video}")
        print("테스트용 영상을 준비해주세요!")
        return

    # 벤치마크 실행
    benchmark = FrameInterpolationBenchmark(input_video)
    benchmark.run_all_tests()

if __name__ == "__main__":
    main()

🎬 입력 영상: final_result.mp4
📊 영상 정보: {'fps': 30.0, 'width': 966, 'height': 500, 'frame_count': 36, 'duration': 1.2}
🚀 프레임 보간 성능 비교 테스트 시작!

🔄 FFmpeg minterpolate 테스트 (목표 FPS: 60)
✅ 완료! 처리시간: 8.00초, 파일크기: 0.16MB

🔄 프레임 블렌딩 테스트 (목표 FPS: 60)
✅ 완료! 처리시간: 1.24초, 파일크기: 0.58MB

🔄 RIFE 테스트 (multiplier: 2)
⚠️  RIFE 저장소 클론 중...
❌ 실패: python3: can't open file '/content/ECCV2022-RIFE/ECCV2022-RIFE/inference_video.py': [Errno 2] No such file or directory


📊 품질 분석 시작...

📊 FFmpeg_minterpolate 품질 분석...
   PSNR: 42.58 ± 0.60
   SSIM: 0.980 ± 0.002

📊 Frame_Blending 품질 분석...
   PSNR: 28.28 ± 5.74
   SSIM: 0.902 ± 0.046
💾 결과 저장: benchmark_results/benchmark_results.json

📊 최종 비교 리포트
방법                   처리시간(초)      파일크기(MB)     PSNR     SSIM    
----------------------------------------------------------------------
FFmpeg_minterpolate  8.00         0.16         42.58    0.980   
Frame_Blending       1.24         0.58         28.28    0.902   

🏆 추천 결과:
   ⚡ 가장 빠름: Frame_Blending (1.24초)
   🎯 최고 품질: FFmpe

In [23]:
def interpolate_missing_frames():
    """빠진 프레임들을 보간으로 채우기"""
    import glob

    images = sorted(glob.glob('output/frame_*.png'))

    cap = cv2.VideoCapture('target.mp4')
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    img = cv2.imread(images[0])
    h, w, _ = img.shape

    out = cv2.VideoWriter('interpolated_normal_speed.mp4',
                         cv2.VideoWriter_fourcc(*'mp4v'), original_fps, (w, h))

    for i in range(len(images) - 1):
        current_frame = cv2.imread(images[i])
        next_frame = cv2.imread(images[i + 1])

        # 현재 프레임 쓰기
        out.write(current_frame)

        # 중간에 9개 프레임 보간
        for j in range(1, 10):
            alpha = j / 10
            blended = cv2.addWeighted(current_frame, 1-alpha, next_frame, alpha, 0)
            out.write(blended)

    # 마지막 프레임
    if images:
        last_frame = cv2.imread(images[-1])
        for _ in range(10):
            out.write(last_frame)

    out.release()
    print("✅ 보간된 정상 속도 영상 완성!")

interpolate_missing_frames()

✅ 보간된 정상 속도 영상 완성!


In [25]:
import cv2
import numpy as np
import time
import subprocess
import os
import json
import matplotlib.pyplot as plt
from datetime import datetime
import psutil
import GPUtil
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import warnings
warnings.filterwarnings('ignore')

class FrameInterpolationBenchmark:
    def __init__(self, input_video_path, output_dir="benchmark_results"):
        """
        프레임 보간 성능 비교 평가 시스템

        Args:
            input_video_path: 테스트할 입력 영상 경로
            output_dir: 결과 저장 디렉토리
        """
        self.input_video = input_video_path
        self.output_dir = output_dir
        self.results = {}

        # 출력 디렉토리 생성
        os.makedirs(output_dir, exist_ok=True)

        # 영상 정보 가져오기
        self.get_video_info()

        print(f"🎬 입력 영상: {input_video_path}")
        print(f"📊 영상 정보: {self.video_info}")

    def get_video_info(self):
        """입력 영상 정보 분석"""
        cap = cv2.VideoCapture(self.input_video)

        self.video_info = {
            'fps': cap.get(cv2.CAP_PROP_FPS),
            'width': int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            'height': int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
            'frame_count': int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
            'duration': cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
        }

        cap.release()

    def monitor_resources(self):
        """시스템 리소스 모니터링"""
        cpu_percent = psutil.cpu_percent(interval=1)
        memory = psutil.virtual_memory()

        gpu_info = []
        try:
            gpus = GPUtil.getGPUs()
            for gpu in gpus:
                gpu_info.append({
                    'name': gpu.name,
                    'utilization': gpu.load * 100,
                    'memory_used': gpu.memoryUsed,
                    'memory_total': gpu.memoryTotal
                })
        except:
            gpu_info = [{'name': 'N/A', 'utilization': 0, 'memory_used': 0, 'memory_total': 0}]

        return {
            'cpu_percent': cpu_percent,
            'memory_percent': memory.percent,
            'memory_used_gb': memory.used / (1024**3),
            'gpu_info': gpu_info
        }

    def test_ffmpeg_minterpolate(self, target_fps=60):
        """FFmpeg minterpolate 테스트"""
        print(f"\n🔄 FFmpeg minterpolate 테스트 (목표 FPS: {target_fps})")

        output_path = os.path.join(self.output_dir, f"ffmpeg_minterpolate_{target_fps}fps.mp4")

        start_time = time.time()
        start_resources = self.monitor_resources()

        cmd = [
            'ffmpeg', '-i', self.input_video,
            '-filter:v', f'minterpolate=fps={target_fps}:mi_mode=mci',
            '-y', output_path
        ]

        try:
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
            processing_time = time.time() - start_time

            if result.returncode == 0:
                end_resources = self.monitor_resources()
                file_size = os.path.getsize(output_path) / (1024**2)  # MB

                self.results['FFmpeg_minterpolate'] = {
                    'status': 'success',
                    'processing_time': processing_time,
                    'output_path': output_path,
                    'file_size_mb': file_size,
                    'start_resources': start_resources,
                    'end_resources': end_resources,
                    'target_fps': target_fps
                }
                print(f"✅ 완료! 처리시간: {processing_time:.2f}초, 파일크기: {file_size:.2f}MB")
            else:
                print(f"❌ 실패: {result.stderr}")
                self.results['FFmpeg_minterpolate'] = {'status': 'failed', 'error': result.stderr}

        except subprocess.TimeoutExpired:
            print("❌ 시간 초과 (300초)")
            self.results['FFmpeg_minterpolate'] = {'status': 'timeout'}
        except Exception as e:
            print(f"❌ 오류: {e}")
            self.results['FFmpeg_minterpolate'] = {'status': 'error', 'error': str(e)}

    def test_rife(self, multiplier=2):
        """RIFE 테스트 (GitHub에서 클론 필요)"""
        print(f"\n🔄 RIFE 테스트 (multiplier: {multiplier})")

        rife_dir = "ECCV2022-RIFE"

        # RIFE 설치 및 설정
        if not os.path.exists(rife_dir):
            print("⚠️  RIFE 저장소 클론 중...")
            try:
                subprocess.run(['git', 'clone', 'https://github.com/megvii-research/ECCV2022-RIFE.git'],
                             check=True, timeout=120)
                print("✅ RIFE 클론 완료")
            except subprocess.CalledProcessError as e:
                print(f"❌ RIFE 클론 실패: {e}")
                self.results['RIFE'] = {'status': 'failed', 'error': f'Clone failed: {e}'}
                return
            except subprocess.TimeoutExpired:
                print("❌ RIFE 클론 시간초과")
                self.results['RIFE'] = {'status': 'failed', 'error': 'Clone timeout'}
                return

        # 필요한 파일들 확인
        inference_script = os.path.join(rife_dir, 'inference_video.py')
        if not os.path.exists(inference_script):
            print("❌ inference_video.py 파일을 찾을 수 없습니다")
            # 대체 스크립트 경로들 확인
            alternative_scripts = [
                os.path.join(rife_dir, 'inference_img.py'),
                os.path.join(rife_dir, 'demo_MiddleBury.py'),
                os.path.join(rife_dir, 'test.py')
            ]

            found_script = None
            for script in alternative_scripts:
                if os.path.exists(script):
                    found_script = script
                    print(f"✅ 대체 스크립트 발견: {script}")
                    break

            if not found_script:
                print("❌ 사용 가능한 RIFE 스크립트를 찾을 수 없습니다")
                self.results['RIFE'] = {'status': 'failed', 'error': 'No inference script found'}
                return

            inference_script = found_script

        # 의존성 설치
        try:
            print("📦 RIFE 의존성 설치 중...")
            original_dir = os.getcwd()
            os.chdir(rife_dir)

            # requirements.txt 설치
            if os.path.exists('requirements.txt'):
                subprocess.run(['pip', 'install', '-r', 'requirements.txt'],
                             capture_output=True, timeout=180)

            # 필수 패키지 설치
            subprocess.run(['pip', 'install', 'torch', 'torchvision', 'opencv-python', 'numpy'],
                         capture_output=True, timeout=180)

            os.chdir(original_dir)

        except Exception as e:
            print(f"⚠️  의존성 설치 중 오류 (계속 진행): {e}")

        # RIFE 실행
        output_path = os.path.join(self.output_dir, f"rife_x{multiplier}.mp4")

        start_time = time.time()
        start_resources = self.monitor_resources()

        # 절대 경로 사용
        abs_input = os.path.abspath(self.input_video)
        abs_output = os.path.abspath(output_path)

        # 여러 가지 명령어 시도
        commands_to_try = [
            # 표준 inference_video.py
            ['python', 'inference_video.py', f'--exp={multiplier}', f'--video={abs_input}', f'--output={abs_output}'],
            # 대체 명령어들
            ['python', 'inference_video.py', '--times', str(multiplier), '--video', abs_input, '--output', abs_output],
            ['python', 'test.py', '--video', abs_input, '--output', abs_output],
        ]

        success = False
        original_dir = os.getcwd()

        try:
            os.chdir(rife_dir)

            for i, cmd in enumerate(commands_to_try):
                print(f"🔄 RIFE 명령어 시도 {i+1}: {' '.join(cmd)}")

                try:
                    result = subprocess.run(cmd, capture_output=True, text=True, timeout=600)

                    if result.returncode == 0:
                        print(f"✅ RIFE 명령어 {i+1} 성공!")
                        success = True
                        break
                    else:
                        print(f"❌ 명령어 {i+1} 실패: {result.stderr[:200]}")

                except subprocess.TimeoutExpired:
                    print(f"❌ 명령어 {i+1} 시간 초과")
                except Exception as e:
                    print(f"❌ 명령어 {i+1} 오류: {e}")

            os.chdir(original_dir)

            processing_time = time.time() - start_time

            if success and os.path.exists(abs_output):
                end_resources = self.monitor_resources()
                file_size = os.path.getsize(abs_output) / (1024**2)

                self.results['RIFE'] = {
                    'status': 'success',
                    'processing_time': processing_time,
                    'output_path': abs_output,
                    'file_size_mb': file_size,
                    'start_resources': start_resources,
                    'end_resources': end_resources,
                    'multiplier': multiplier
                }
                print(f"✅ RIFE 완료! 처리시간: {processing_time:.2f}초, 파일크기: {file_size:.2f}MB")
            else:
                print("❌ RIFE 실행 실패 - 모든 명령어 시도 완료")
                self.results['RIFE'] = {'status': 'failed', 'error': 'All command attempts failed'}

        except Exception as e:
            os.chdir(original_dir)
            print(f"❌ RIFE 실행 중 오류: {e}")
            self.results['RIFE'] = {'status': 'error', 'error': str(e)}

    def test_frame_blending(self, target_fps=60):
        """간단한 프레임 블렌딩 테스트 (기준선)"""
        print(f"\n🔄 프레임 블렌딩 테스트 (목표 FPS: {target_fps})")

        output_path = os.path.join(self.output_dir, f"frame_blending_{target_fps}fps.mp4")

        start_time = time.time()
        start_resources = self.monitor_resources()

        try:
            cap = cv2.VideoCapture(self.input_video)

            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, target_fps,
                                (self.video_info['width'], self.video_info['height']))

            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)

            cap.release()

            # 프레임 간 블렌딩으로 보간
            interpolation_factor = target_fps / self.video_info['fps']

            for i in range(len(frames) - 1):
                out.write(frames[i])

                # 중간 프레임들 생성
                num_inter_frames = int(interpolation_factor) - 1
                for j in range(1, num_inter_frames + 1):
                    alpha = j / (num_inter_frames + 1)
                    blended = cv2.addWeighted(frames[i], 1-alpha, frames[i+1], alpha, 0)
                    out.write(blended)

            if frames:
                out.write(frames[-1])

            out.release()

            processing_time = time.time() - start_time
            end_resources = self.monitor_resources()
            file_size = os.path.getsize(output_path) / (1024**2)

            self.results['Frame_Blending'] = {
                'status': 'success',
                'processing_time': processing_time,
                'output_path': output_path,
                'file_size_mb': file_size,
                'start_resources': start_resources,
                'end_resources': end_resources,
                'target_fps': target_fps
            }
            print(f"✅ 완료! 처리시간: {processing_time:.2f}초, 파일크기: {file_size:.2f}MB")

        except Exception as e:
            print(f"❌ 오류: {e}")
            self.results['Frame_Blending'] = {'status': 'error', 'error': str(e)}

    def calculate_quality_metrics(self, method_name, output_path):
        """품질 지표 계산 (PSNR, SSIM)"""
        print(f"\n📊 {method_name} 품질 분석...")

        try:
            # 원본과 결과 영상에서 샘플 프레임 추출
            original_frames = self.extract_sample_frames(self.input_video, num_samples=10)
            result_frames = self.extract_sample_frames(output_path, num_samples=10)

            if len(original_frames) == 0 or len(result_frames) == 0:
                return None

            psnr_scores = []
            ssim_scores = []

            # 크기 맞추기
            min_samples = min(len(original_frames), len(result_frames))

            for i in range(min_samples):
                orig = original_frames[i]
                result = result_frames[i]

                # 크기 조정
                if orig.shape != result.shape:
                    result = cv2.resize(result, (orig.shape[1], orig.shape[0]))

                # 그레이스케일 변환
                orig_gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
                result_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

                # PSNR 계산
                psnr_val = psnr(orig_gray, result_gray, data_range=255)
                psnr_scores.append(psnr_val)

                # SSIM 계산
                ssim_val = ssim(orig_gray, result_gray, data_range=255)
                ssim_scores.append(ssim_val)

            quality_metrics = {
                'avg_psnr': np.mean(psnr_scores),
                'avg_ssim': np.mean(ssim_scores),
                'psnr_std': np.std(psnr_scores),
                'ssim_std': np.std(ssim_scores)
            }

            self.results[method_name]['quality_metrics'] = quality_metrics
            print(f"   PSNR: {quality_metrics['avg_psnr']:.2f} ± {quality_metrics['psnr_std']:.2f}")
            print(f"   SSIM: {quality_metrics['avg_ssim']:.3f} ± {quality_metrics['ssim_std']:.3f}")

            return quality_metrics

        except Exception as e:
            print(f"❌ 품질 분석 실패: {e}")
            return None

    def extract_sample_frames(self, video_path, num_samples=10):
        """영상에서 샘플 프레임 추출"""
        cap = cv2.VideoCapture(video_path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if frame_count == 0:
            cap.release()
            return []

        frames = []
        step = max(1, frame_count // num_samples)

        for i in range(0, frame_count, step):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if ret:
                frames.append(frame)
                if len(frames) >= num_samples:
                    break

        cap.release()
        return frames

    def run_all_tests(self):
        """모든 테스트 실행"""
        print("🚀 프레임 보간 성능 비교 테스트 시작!")
        print("=" * 60)

        # 1. FFmpeg minterpolate
        self.test_ffmpeg_minterpolate(target_fps=60)

        # 2. Frame Blending (기준선)
        self.test_frame_blending(target_fps=60)

        # 3. RIFE (조건부)
        if self.should_test_rife():
            self.test_rife(multiplier=2)
        else:
            print("\n⚠️  RIFE 테스트 건너뜀 (의존성 없음)")

        # 품질 분석
        print("\n" + "=" * 60)
        print("📊 품질 분석 시작...")

        for method, result in self.results.items():
            if result.get('status') == 'success':
                self.calculate_quality_metrics(method, result['output_path'])

        # 결과 저장 및 시각화
        self.save_results()
        self.create_comparison_report()

    def should_test_rife(self):
        """RIFE 테스트 가능 여부 확인"""
        try:
            # PyTorch 설치 여부 확인
            import torch
            return True
        except ImportError:
            return False

    def save_results(self):
        """결과를 JSON 파일로 저장"""
        results_file = os.path.join(self.output_dir, "benchmark_results.json")

        # datetime 객체들을 문자열로 변환
        serializable_results = {}
        for method, result in self.results.items():
            serializable_results[method] = {}
            for key, value in result.items():
                if isinstance(value, (dict, list, str, int, float, bool)) or value is None:
                    serializable_results[method][key] = value
                else:
                    serializable_results[method][key] = str(value)

        with open(results_file, 'w', encoding='utf-8') as f:
            json.dump({
                'timestamp': datetime.now().isoformat(),
                'input_video_info': self.video_info,
                'results': serializable_results
            }, f, indent=2, ensure_ascii=False)

        print(f"💾 결과 저장: {results_file}")

    def create_comparison_report(self):
        """비교 리포트 생성"""
        print("\n" + "=" * 60)
        print("📊 최종 비교 리포트")
        print("=" * 60)

        successful_methods = {k: v for k, v in self.results.items() if v.get('status') == 'success'}

        if not successful_methods:
            print("❌ 성공한 테스트가 없습니다!")
            return

        # 성능 비교 테이블
        print(f"{'방법':<20} {'처리시간(초)':<12} {'파일크기(MB)':<12} {'PSNR':<8} {'SSIM':<8}")
        print("-" * 70)

        for method, result in successful_methods.items():
            processing_time = result.get('processing_time', 0)
            file_size = result.get('file_size_mb', 0)

            quality = result.get('quality_metrics', {})
            psnr_val = quality.get('avg_psnr', 0)
            ssim_val = quality.get('avg_ssim', 0)

            print(f"{method:<20} {processing_time:<12.2f} {file_size:<12.2f} {psnr_val:<8.2f} {ssim_val:<8.3f}")

        # 추천 결과
        print("\n🏆 추천 결과:")

        # 속도 기준 최고
        fastest = min(successful_methods.items(), key=lambda x: x[1].get('processing_time', float('inf')))
        print(f"   ⚡ 가장 빠름: {fastest[0]} ({fastest[1].get('processing_time', 0):.2f}초)")

        # 품질 기준 최고 (SSIM)
        quality_methods = {k: v for k, v in successful_methods.items() if v.get('quality_metrics')}
        if quality_methods:
            best_quality = max(quality_methods.items(),
                             key=lambda x: x[1].get('quality_metrics', {}).get('avg_ssim', 0))
            ssim_score = best_quality[1].get('quality_metrics', {}).get('avg_ssim', 0)
            print(f"   🎯 최고 품질: {best_quality[0]} (SSIM: {ssim_score:.3f})")

        # 파일 크기 최적화
        smallest = min(successful_methods.items(), key=lambda x: x[1].get('file_size_mb', float('inf')))
        print(f"   💾 최소 용량: {smallest[0]} ({smallest[1].get('file_size_mb', 0):.2f}MB)")

# 사용 예시
def main():
    # 테스트할 영상 경로
    input_video = "final_result.mp4"  # 여기에 테스트할 영상 경로 입력

    if not os.path.exists(input_video):
        print(f"❌ 입력 영상을 찾을 수 없습니다: {input_video}")
        print("테스트용 영상을 준비해주세요!")
        return

    # 벤치마크 실행
    benchmark = FrameInterpolationBenchmark(input_video)
    benchmark.run_all_tests()

if __name__ == "__main__":
    main()

🎬 입력 영상: final_result.mp4
📊 영상 정보: {'fps': 30.0, 'width': 966, 'height': 500, 'frame_count': 36, 'duration': 1.2}
🚀 프레임 보간 성능 비교 테스트 시작!

🔄 FFmpeg minterpolate 테스트 (목표 FPS: 60)
✅ 완료! 처리시간: 6.47초, 파일크기: 0.16MB

🔄 프레임 블렌딩 테스트 (목표 FPS: 60)
✅ 완료! 처리시간: 1.21초, 파일크기: 0.58MB

🔄 RIFE 테스트 (multiplier: 2)
📦 RIFE 의존성 설치 중...
🔄 RIFE 명령어 시도 1: python inference_video.py --exp=2 --video=/content/final_result.mp4 --output=/content/benchmark_results/rife_x2.mp4
❌ 명령어 1 실패: Traceback (most recent call last):
  File "/content/ECCV2022-RIFE/inference_video.py", line 91, in <module>
    from model.RIFE_HDv2 import Model
ModuleNotFoundError: No module named 'model.RIFE_HDv2'
🔄 RIFE 명령어 시도 2: python inference_video.py --times 2 --video /content/final_result.mp4 --output /content/benchmark_results/rife_x2.mp4
❌ 명령어 2 실패: usage: inference_video.py [-h] [--video VIDEO] [--output OUTPUT] [--img IMG]
                          [--montage] [--model MODELDIR] [--fp16] [--UHD]
                          [--scale SCALE

In [32]:
def change_video_speed_colab(input_video, output_video, speed=1.0):
    """
    오디오 없는 영상 속도 조절 (느리게/빠르게) - Colab 용
    Args:
        input_video (str): 입력 파일
        output_video (str): 출력 파일
        speed (float): 속도 배수 (예: 0.25 = 느리게, 4.0 = 빠르게)
    """
    print(f"🎞️ 영상 속도 {speed}배 {'빠르게' if speed > 1 else '느리게'}: {input_video}")

    # setpts 공식: 느리게는 큰 값, 빠르게는 작은 값
    setpts_value = speed
    cmd = [
        'ffmpeg', '-i', input_video,
        '-filter:v', f'setpts={setpts_value:.3f}*PTS',
        '-y', output_video
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode == 0:
        size = os.path.getsize(output_video) / (1024**2)
        print(f"✅ 완료: {output_video} ({size:.1f}MB)")
    else:
        print(f"❌ 실패: {result.stderr}")

##ffmpeg승


In [34]:
# 예: final_result.mp4 를 0.25배 ~ 4배 속도로 각각 생성
batch_change_speeds("/content/benchmark_results/frame_blending_60fps.mp4", speeds=[0.25, 0.5, 1.0, 2.0, 4.0])

⚙️ 원본: /content/benchmark_results/frame_blending_60fps.mp4
🎞️ 영상 속도 0.25배 느리게: /content/benchmark_results/frame_blending_60fps.mp4
✅ 완료: /content/benchmark_results/frame_blending_60fps_0.25x.mp4 (0.1MB)
🎞️ 영상 속도 0.5배 느리게: /content/benchmark_results/frame_blending_60fps.mp4
✅ 완료: /content/benchmark_results/frame_blending_60fps_0.5x.mp4 (0.1MB)
🎞️ 영상 속도 1.0배 느리게: /content/benchmark_results/frame_blending_60fps.mp4
✅ 완료: /content/benchmark_results/frame_blending_60fps_1.0x.mp4 (0.2MB)
🎞️ 영상 속도 2.0배 빠르게: /content/benchmark_results/frame_blending_60fps.mp4
✅ 완료: /content/benchmark_results/frame_blending_60fps_2.0x.mp4 (0.3MB)
🎞️ 영상 속도 4.0배 빠르게: /content/benchmark_results/frame_blending_60fps.mp4
✅ 완료: /content/benchmark_results/frame_blending_60fps_4.0x.mp4 (0.4MB)
